### Dependencies

In [22]:
# Import Dependencies
import plotly.graph_objects as go

In [1]:
def parse_coordinates(coordinates):
    # Convert DMS coordinates to decimal degrees
    latitudes, longitudes = [], []
    for lat_str, lon_str in coordinates:
        lat_deg = round(float(lat_str[:2]) + float(lat_str[2:4]) / 60, 5)
        lon_deg = round(float(lon_str[:3]) + float(lon_str[3:5]) / 60, 5)
        if lat_str.endswith('S'):
            lat_deg *= -1
        if lon_str.endswith('W'):
            lon_deg *= -1
        latitudes.append(lat_deg)
        longitudes.append(lon_deg)
    return latitudes, longitudes

In [2]:
def round_latlon(latitude, longitude):
    # round the latitude, longitude coordinate to 5 decimal places
    # Copy pasting from gmaps gives 10+ decimal places
    return round(latitude, 5), round(longitude, 5)

In [3]:
def create_latlon_list(rwy_polygon_points):
    """Returns a list of latitude and list of longitude from an input of a list of the points bounding a runway/polygon. """
    
    lat_list = [pt[0] for pt in rwy_polygon_points] + [rwy_polygon_points[0][0]]
    lon_list = [pt[1] for pt in rwy_polygon_points] + [rwy_polygon_points[0][1]]
    return lat_list, lon_list

In [4]:
def add_airport(fig, name, code, lat, lon, color, size, legendgroup, showlegend=True):
    """
    Adds an airport marker with a text label and hover popup.
    
    Parameters:
    - fig: the plotly figure object
    - name: full airport name (e.g., "Seletar Airport")
    - code: ICAO code (e.g., "WSSL")
    - lat, lon: coordinates
    - color: marker color
    - size: marker size
    - legendgroup: group name for legend grouping
    - showlegend: whether to display this item in the legend
    """
    fig.add_trace(go.Scattermap(
        lat=[lat],
        lon=[lon],
        mode='markers+text',
        marker=dict(size=size, color=color, symbol='circle'),
        textposition='top right',
        name=f'{name} ({code})',
        hovertext=code,
        legendgroup=legendgroup,
        legendgrouptitle_text=name,
        showlegend=showlegend,
        hovertemplate='📍 %{fullData.name}<extra></extra>'
    ))


def add_runway(fig, name, lat_list, lon_list, fillcolor, linecolor, linewidth, legendgroup, legendgrouptitle_text = None, showlegend=True, ):
    """
    Adds a filled polygon to represent a runway.
    
    Parameters:
    - fig: the plotly figure object
    - name: runway label (e.g., "RWY 03/21")
    - lat_list, lon_list: lists of coordinates outlining the runway
    - fillcolor: polygon fill
    - linecolor: boundary line color
    - linewidth: boundary line width
    - legendgroup: group name for legend grouping
    - showlegend: whether to display this item in the legend
    """
    fig.add_trace(go.Scattermap(
        lat=lat_list,
        lon=lon_list,
        mode='lines',
        fill='toself',
        fillcolor=fillcolor,
        line=dict(color=linecolor, width=linewidth),
        name=name,
        legendgroup=legendgroup,
        legendgrouptitle_text=legendgrouptitle_text,
        showlegend=showlegend
    ))

In [ ]:
from matplotlib.colors import to_rgba

def css_to_rgba(color: str, opacity: float) -> str:
    """
    Converts a CSS color name (e.g. 'skyblue') or hex string to an rgba() string with the given opacity.

    Parameters:
    - color: CSS color name or hex (e.g. 'orange', '#ffcc00')
    - opacity: Float between 0.0 and 1.0

    Returns:
    - 'rgba(R, G, B, A)' formatted string
    """
    r, g, b, _ = to_rgba(color)
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return f'rgba({r}, {g}, {b}, {opacity})'


def add_sector(fig, name, coordinates, fillcolor, linecolor, opacity=0.3, linewidth=1, 
               legendgroup=None, legendgrouptitle_text=None, legendrank=None, showlegend=True):
    """
    Adds an airspace sector polygon to a Plotly map.

    Parameters:
    - fig: Plotly figure object
    - name: Legend label
    - coordinates: List of (lat_dms, lon_dms) tuples (e.g., '013112N', '1035936E')
    - fillcolor: CSS color name or rgba() string
    - linecolor: Polygon outline color
    - opacity: Polygon fill transparency (0.0 - 1.0)
    - linewidth: Outline thickness
    - showlegend: Whether to show in legend
    - legendgroup: Optional legend group
    - legendrank: Controls order in legend (lower = higher up)
    - legendgrouptitle_text: Optional group title for legend section
    """

    # Convert DMS to decimal degrees
    lat_list, lon_list = parse_coordinates(coordinates)

    # Round coordinates
    lat_list = [round_latlon(lat, lon)[0] for lat, lon in zip(lat_list, lon_list)]
    lon_list = [round_latlon(lat, lon)[1] for lat, lon in zip(lat_list, lon_list)]

    # Convert CSS color to rgba if needed
    if ',' not in fillcolor and not fillcolor.startswith('rgba'):
        fillcolor = css_to_rgba(fillcolor, opacity)

    # Add the sector polygon trace
    fig.add_trace(go.Scattermap(
        lat=lat_list,
        lon=lon_list,
        mode='lines',
        fill='toself',
        fillcolor=fillcolor,
        line=dict(color=linecolor, width=linewidth),
        name=name,
        legendgroup=legendgroup,
        legendgrouptitle_text=legendgrouptitle_text,
        legendrank=legendrank,
        hoverinfo='skip',
        showlegend=showlegend,
    ))

In [2]:
WSSS_coordinates = [('012133N', '1035922E')]

SG_FIR_coordinates = [
    ('082500N', '1163000E'), 
    ('025050N', '1091629E'), 
    ('045700N', '1081619E'), 
    ('050012N', '1080132E'), 
    ('045904N', '1075525E'), 
    ('045203N', '1074625E'), 
    ('043820N', '1073315E'), 
    ('041312N', '1071743E'), 
    ('033045N', '1055130E'), 
    ('031727N', '1052959E'), 
    ('031453N', '1052619E'), 
    ('025010N', '1051210E'), 
    ('024348N', '1050854E'), 
    ('023641N', '1051311E'), 
    ('021838N', '1052205E'), 
    ('011947N', '1044606E'), 
    ('012921N', '1043441E'), 
    ('011800N', '1043000E'), 
    ('011500N', '1040000E'), 
    ('010800N', '1034500E'), 
    ('011046N', '1034015E'), 
    ('011200N', '1033900E'), 
    ('011408N', '1033142E'), 
    ('011700N', '1033600E'), 
    ('012608N', '1034055E'), # National boundary of Singapore/Malaysia
    ('012730N', '1034326E'), # National boundary of Singapore/Malaysia
    ('012640N', '1034540E'), # National boundary of Singapore/Malaysia
    ('012832N', '1034809E'), # National boundary of Singapore/Malaysia
    ('012557N', '1035320E'), # National boundary of Singapore/Malaysia
    ('012513N', '1040006E'), # National boundary of Singapore/Malaysia
    ('012638N', '1040222E'), # National boundary of Singapore/Malaysia
    ('012421N', '1040528E'), # National boundary of Singapore/Malaysia
    ('012127N', '1040440E'), # National boundary of Singapore/Malaysia
    ('011937N', '1040551E'), # National boundary of Singapore/Malaysia
    ('012000N', '1042000E'), 
    ('023600N', '1044500E'), 
    ('034000N', '1034000E'), 
    ('045000N', '1034400E'), 
    ('064500N', '1024000E'), 
    ('070000N', '1030000E'), 
    ('070000N', '1080000E'), 
    ('103000N', '1140000E'), 
    ('082500N', '1163000E')
]

KL_FIR_vested1_coordinates = [
    ('023600N', '1044500E'),
    ('023200N', '1050000E'),
    ('060000N', '1050000E'), 
    ('060000N', '1030500E'), 
    ('045000N', '1034400E'), 
    ('034000N', '1034000E'), 
    ('023600N', '1044500E')
]

KL_FIR_vested2_coordinates = [
    ('023200N', '1050000E'),
    ('060000N', '1050000E'), 
    ('060000N', '1132000E'), 
    ('025050N', '1091629E'), 
    ('045700N', '1081619E'), 
    ('050012N', '1080132E'), 
    ('045904N', '1075525E'), 
    ('045203N', '1074625E'), 
    ('043820N', '1073315E'), 
    ('041312N', '1071743E'), 
    ('033045N', '1055130E'), 
    ('031727N', '1052959E'), 
    ('031453N', '1052619E'), 
    ('025010N', '1051210E'), 
    ('024348N', '1050854E'), 
    ('023641N', '1051311E'),
    ('022715N', '1051750E'),
    ('023200N', '1050000E'),
]

JAKARTA_FIR_delegated_coordinates = [
    ('031727N', '1052959E'),
    ('012450N', '1061648E'),
    ('001030N', '1045656E'),
    ('000000N', '1050340E'),
    ('000000N', '1044330E'),
    ('001113S', '1042205E'), # Arc of airspace
    ('001620S', '1035827E'), # Arc of airspace
    ('001458S', '1033419E'), # Arc of airspace
    ('000714S', '1031124E'), # Arc of airspace
    ('000618N', '1025123E'), # Arc of airspace
    ('002442N', '1023541E'), # Arc of airspace
    ('004636N', '1022528E'), # Arc of airspace
    ('011026N', '1022126E'), # Arc of airspace
    ('013430N', '1022353E'),
    ('011300N', '1033000E'),
    ('011408N', '1033142E'),
    ('011200N', '1033900E'),
    ('011046N', '1034015E'),
    ('010800N', '1034500E'),
    ('011500N', '1040000E'),
    ('011800N', '1043000E'),
    ('012921N', '1043441E'),
    ('011947N', '1044606E'),
    ('021838N', '1052205E'),
    ('023641N', '1051311E'),
    ('024348N', '1050854E'),
    ('025010N', '1051210E'),
    ('031727N', '1052959E')
]

SG_JHR_airspace_complex_coordinates = [
    ('022600N', '1025605E'),
    ('022600N', '1043400E'),
    ('004300N', '1043400E'),
    ('004300N', '1025605E'),
    ('022600N', '1025605E')
]

waypoints_all = {
    'ABVIP': ('010008N', '1035032E'),
    'ABVON': ('012028N', '1035827E'),
    'ADNIK': ('011651N', '1035655E'),
    'ADPON': ('011203N', '1040514E'),
    # 'ADPON': ('010108N', '1035808E'),
    'AGROT': ('010108N', '1035808E'),
    'AGVAR': ('014719N', '1034145E'),
    'AKDAT': ('032923N', '1054917E'),
    'AKIPO': ('011356N', '1035542E'),
    'AKMET': ('015355N', '1034339E'),
    'AKMON': ('081254N', '1101306E'),
    'AKOMA': ('014522N', '1035443E'),
    'AKVOM': ('005620N', '1041514E'),
    'ANBUS': ('011554N', '1032100E'),
    'ANITO': ('001700S', '1045200E'),
    'ANUMA': ('011053N', '1035424E'),
    'APIPA': ('010618N', '1035228E'),
    'ARAMA': ('013654N', '1030712E'),
    'AROSO': ('020846N', '1032421E'),
    'ASISU': ('055906N', '1132046E'),
    'ASITI': ('004906N', '1035042E'),
    'ASOMI': ('010142N', '1040207E'),
    'ASUNA': ('005948N', '1030954E'),
    'ATLEX': ('010302N', '1033331E'),
    'ATLIR': ('011120N', '1035208E'),
    'ATPOM': ('002425N', '1052114E'),
    'ATRUM': ('013256N', '1040057E'),
    'AVLUB': ('003112S', '1042501E'),
    'AVPIV': ('011207N', '1035349E'),
    'BAVAL': ('004518N', '1040242E'),
    'BETBA': ('013302N', '1035331E'),
    'BIBVI': ('024336N', '1040618E'),
    'BIDAG': ('073101N', '1135544E'),
    'BIDUS': ('013554N', '1035755E'),
    'BIKTA': ('024337N', '1034308E'),
    'BIMOS': ('011512N', '1035815E'),
    'BIPOP': ('013122N', '1041018E'),
    'BISOV': ('004229N', '1025214E'),
    'BISUT': ('011218N', '1035701E'),
    'BITAM': ('010813N', '1040757E'),
    'BOBAG': ('010230N', '1032954E'),
    'BOBOB': ('022206N', '1070558E'),
    'BONSU': ('011928N', '1033710E'),
    'BOPVA': ('025303N', '1051349E'),
    'BUNTO': ('024200N', '1060000E'),
    'BUVAL': ('033622N', '1034341E'),
    'DAKIX': ('070854N', '1145054E'),
    'DAMOG': ('041225N', '1050014E'),
    'DODSO': ('012225N', '1061402E'),
    'DOLOX': ('044841N', '1052247E'),
    'DOVAN': ('011938N', '1041249E'),
    'DOVOL': ('033047N', '1034923E'),
    'DOWON': ('011957N', '1043048E'),
    'DUBOT': ('010846N', '1040103E'),
    'DUBSA': ('034901N', '1044540E'),
    'DUDIS': ('070000N', '1064836E'),
    'DUMUP': ('005430N', '1035516E'),
    'EGOLO': ('031934N', '1040047E'),
    'EGORA': ('013621N', '1040607E'),
    'ELALO': ('041240N', '1043329E'),
    'ELALU': ('013440N', '1040524E'),
    'ELBEB': ('012845N', '1040254E'),
    'ELBEX': ('013149N', '1040314E'),
    'ELGAP': ('012820N', '1040146E'),
    'ELGOR': ('033014N', '1054818E'),
    'ELMIN': ('012550N', '1040141E'),
    'EMRIX': ('012606N', '1041040E'),
    'EMSIB': ('005911N', '1035419E'),
    'EMSUX': ('024647N', '1051026E'),
    'EMTAP': ('011656N', '1035657E'),
    'ENLES': ('010932N', '1035350E'),
    'ENPUX': ('002859S', '1043434E'),
    'ENREP': ('045224N', '1041442E'),
    'ENSUN': ('012603N', '1040048E'),
    'ENVUM': ('011535N', '1040552E'),
    'ERVIV': ('010445N', '1041013E'),
    'ERVOT': ('011120N', '1035436E'),
    'ESBIT': ('012212N', '1040009E'),
    'ESBUM': ('045210N', '1042830E'),
    'ESLUX': ('011844N', '1035840E'),
    'ESPOB': ('070000N', '1053318E'),
    'EXOMO': ('010425N', '1040933E'),
    'GIXEM': ('004920N', '1042539E'),
    'GOTGA': ('012013N', '1044200E'),
    'GULGU': ('040141N', '1084242E'),
    'GULIB': ('041714N', '1110633E'),
    'GUMPU': ('013000N', '1034243E'),
    'GUNUD': ('011042N', '1050618E'),
    'GURES': ('002814N', '1043835E'),
    'GUTUP': ('045911N', '1075603E'),
    'HOSBA': ('011948N', '1042418E'),
    'IBASU': ('005751N', '1033410E'),
    'IBIVA': ('011351N', '1035637E'),
    'IBIXU': ('011621N', '1035740E'),
    'IDBUD': ('001454N', '1050139E'),
    'IDEMO': ('025431N', '1040603E'),
    'IDKIV': ('005652N', '1041333E'),
    'IDMAS': ('004900N', '1041848E'),
    'IDSEL': ('032432N', '1035544E'),
    'IDUNA': ('012306N', '1035934E'),
    'IDURO': ('012640N', '1040104E'),
    'IDVAS': ('012935N', '1040218E'),
    'IGARI': ('065612N', '1033506E'),
    'IGNON': ('010847N', '1041257E'),
    'IGOSI': ('005645N', '1040644E'),
    'IGULA': ('013232N', '1040333E'),
    'IGUTU': ('001331S', '1041857E'),
    'IKIRO': ('000849N', '1044420E'),
    'IKUKO': ('054512N', '1031324E'),
    'IKUMI': ('055338N', '1035509E'),
    'INVUB': ('002749N', '1051530E'),
    'IPDOL': ('045111N', '1035920E'),
    'IPNAK': ('013712N', '1040531E'),
    'IPRIX': ('070000N', '1040754E'),
    'IRPUG': ('005813N', '1040127E'),
    'IRSAB': ('024349N', '1054359E'),
    'IRTAD': ('010326N', '1041147E'),
    'ISDEB': ('024440N', '1063011E'),
    'ISGIL': ('004246N', '1031257E'),
    'ISNOM': ('010629N', '1035826E'),
    'JUNHA': ('005413N', '1043052E'),
    'KAKSA': ('011703N', '1035758E'),
    'KANLA': ('034556N', '1043606E'),
    'KARTO': ('011124N', '1053343E'),
    'KASPO': ('011507N', '1035709E'),
    'KETOD': ('031042N', '1040942E'),
    'KEXAS': ('011019N', '1044818E'),
    'KEXOL': ('043930N', '1040942E'),
    'KIBOL': ('025224N', '1042818E'),
    'KILOT': ('030217N', '1044023E'),
    'KIMER': ('011106N', '1035527E'),
    'KIRDA': ('000009N', '1045934E'),
    'LAGOT': ('071632N', '1113243E'),
    'LAGUS': ('011915N', '1035854E'),
    'LAPOL': ('012622N', '1034435E'),
    'LASIN': ('011538N', '1035722E'),
    'LAVAX': ('010950N', '1042714E'),
    'LAXOR': ('094937N', '1144829E'),
    'LEBIN': ('031438N', '1060604E'),
    'LEDOX': ('011642N', '1035651E'),
    'LEGOL': ('012053N', '1034723E'),
    'LELIB': ('012729N', '1032450E'),
    'LELON': ('011244N', '1035609E'),
    'LENDA': ('024124N', '1043932E'),
    'LEPNA': ('010648N', '1035339E'),
    'LETGO': ('011411N', '1035548E'),
    'LIDVA': ('010506N', '1035339E'),
    'LIGVU': ('034417N', '1061859E'),
    'LIPRO': ('025342N', '1051128E'),
    'LUSMO': ('033341N', '1065534E'),
    'LUXOL': ('011803N', '1035823E'),
    'MABAL': ('032826N', '1051236E'),
    'MABLI': ('041717N', '1061247E'),
    'MANIM': ('031430N', '1040554E'),
    'MASBO': ('020248N', '1025251E'),
    'MASNI': ('012037N', '1033746E'),
    'MELAS': ('070518N', '1080912E'),
    'MIBEL': ('012351N', '1020816E'),
    'MOLVO': ('012955N', '1040227E'),
    'MOXIB': ('012933N', '1040315E'),
    'MUMDU': ('010521N', '1042714E'),
    'MUMSO': ('034420N', '1053213E'),
    'NIVAM': ('023650N', '1040228E'),
    'NIXEB': ('013943N', '1061040E'),
    'NODIN': ('081100N', '1161142E'),
    'NOPAT': ('042313N', '1044756E'),
    'NUFFA': ('025341N', '1033829E'),
    'NYLON': ('013657N', '1040624E'),
    'OBDAB': ('031153N', '1040538E'),
    'OBDOS': ('002503N', '1065551E'),
    'OBGET': ('012307N', '1064531E'),
    'ODONO': ('063614N', '1030129E'),
    'OLKIT': ('045010N', '1115118E'),
    'OLMUT': ('030306N', '1053558E'),
    'OLNUB': ('011110N', '1035147E'),
    'OMDUD': ('005847N', '1035714E'),
    'OMKOM': ('013112N', '1035910E'),
    'OMLIV': ('025512N', '1062812E'),
    'OSERU': ('024450N', '1054334E'),
    'OTLAL': ('004209N', '1053052E'),
    'OTLON': ('030752N', '1042006E'),
    'PADLI': ('030918N', '1033133E'),
    'PALGA': ('011059N', '1034759E'),
    'PAMSI': ('010459N', '1034845E')
}

STAR_SID_waypoints = {
    'ABVIP': ('010008N', '1035032E'),
    'ADPON': ('011203N', '1040514E'),
    # 'ADPON': ('010108N', '1035808E'),
    'AGROT': ('010108N', '1035808E'),
    'AGVAR': ('014719N', '1034145E'),
    'AKMET': ('015355N', '1034339E'),
    'AKOMA': ('014522N', '1035443E'),
    'ALFA' : ('013033N', '1034942E'),
    'ANITO': ('001700S', '1045200E'),
    'ARAMA': ('013654N', '1030712E'),
    'AROSO': ('020846N', '1032421E'),
    'ASITI': ('004906N', '1035042E'),
    'ASOMI': ('010142N', '1040207E'),
    'ASUNA': ('005948N', '1030954E'),
    'ATLEX': ('010302N', '1033331E'),
    'ATRUM': ('013256N', '1040057E'),
    'BETBA': ('013302N', '1035331E'),
    'BIBVI': ('024336N', '1040618E'),
    'BIDUS': ('013554N', '1035755E'),
    'BIPOP': ('013122N', '1041018E'),
    'BISOV': ('004229N', '1025214E'),
    'BITAM': ('010813N', '1040757E'),
    'BOBAG': ('010230N', '1032954E'),
    'BOKIP': ('010421N', '1034353E'),
    'BTM'  : ('010813N', '1040758E'),
    'DODSO': ('012225N', '1061402E'),
    'DOVAN': ('011938N', '1041249E'),
    'DUBOT': ('010846N', '1040103E'),
    'DUMUP': ('005430N', '1035516E'),
    'ELALO': ('041240N', '1043329E'),
    'EMRIX': ('012606N', '1041040E'),
    'ERVIV': ('010445N', '1041013E'),
    'GIXEM': ('004920N', '1042539E'),
    'GOTGA': ('012013N', '1044200E'),
    'GUMPU': ('013000N', '1034243E'),
    'GUNUD': ('011042N', '1050618E'),
    'GURES': ('002814N', '1043835E'),
    'HOSBA': ('011948N', '1042418E'),
    'IBASU': ('005751N', '1033410E'),
    'IBIVA': ('011351N', '1035637E'),
    'IBIXU': ('011621N', '1035740E'),
    'IDBUD': ('001454N', '1050139E'),
    'IDKIV': ('005652N', '1041333E'),
    'IGNON': ('010847N', '1041257E'),
    'IGOSI': ('005645N', '1040644E'),
    'IKIRO': ('000849N', '1044420E'),
    'ISGIL': ('004246N', '1031257E'),
    'ISNOM': ('010629N', '1035826E'),
    'KANLA': ('034556N', '1043606E'),
    'KARTO': ('011124N', '1053343E'),
    'KEXAS': ('011019N', '1044818E'),
    'KILOT': ('030217N', '1044023E'),
    'KIRDA': ('000009N', '1045934E'),
    'LAMA' : ('013150N', '1035850E'), #Added from ENR 3.6 holding list
    'LAVAX': ('010950N', '1042714E'),
    'LEDOX': ('011642N', '1035651E'),
    'LELIB': ('012729N', '1032450E'),
    'LETGO': ('011411N', '1035548E'),
    'MABAL': ('032826N', '1051236E'),
    'MASBO': ('020248N', '1025251E'),
    'MIBEL': ('012351N', '1020816E'),
    'MOLVO': ('012955N', '1040227E'),
    'MOXIB': ('012933N', '1040315E'),
    'MUMDU': ('010521N', '1042714E'),
    'NYLON': ('013657N', '1040624E'),
    'PALGA': ('011059N', '1034759E'),
    'PAMSI': ('010459N', '1034845E'),
    'PASPU': ('015915N', '1040618E'),
    'PIBAP': ('023023N', '1040618E'),
    'POSUB': ('012725N', '1040748E'),
    'POVEB': ('011344N', '1040130E'),
    'PU'   : ('012524N', '1035600E'),
    'REMES': ('004342N', '1035735E'),
    'REPOV': ('001623N', '1040300E'),
    'RWY 02R DER': ('012122N', '1040051E'),
    'RWY 02C DER': ('012145N', '1035957E'),
    'RWY 02L DER': ('012305N', '1035933E'),
    'RWY 20C DER': ('011942N', '1035905E'),
    'RWY 20R DER': ('012047N', '1035835E'),
    'RWY 20L DER': ('011919N', '1035959E'),
    'SABKA': ('015051N', '1031713E'),
    'SALRU': ('011701N', '1040802E'),
    'SAMKO': ('010530N', '1035255E'),
    'SANAT': ('010749N', '1035930E'),
    'SEBVO': ('011259N', '1044028E'),
    'SJ'   : ('011321N', '1035115E'),
    'SINJON': ('011321N', '1035115E'),
    'SURGA': ('003657S', '1063119E'),
    'TAROS': ('004200N', '1021612E'),
    'TEBUN': ('011455N', '1031557E'),
    'TOMAN': ('012147N', '1054717E'),
    'TUSPI': ('003301N', '1040959E'), #Added from ENR 3.6 holding list
    'UGEBO': ('003813N', '1052432E'),
    'UKIBO': ('011758N', '1035924E'),
    'UPTEL': ('005925N', '1040730E'),
    'VAMPO': ('005833N', '1032525E'),
    'VANBU': ('010643N', '1042740E'),
    'VASTI': ('004320N', '1043406E'),
    'VEBMA': ('012030N', '1045332E'),
    'VEXEL': ('005904N', '1034254E'),
    'VIBOG': ('004310N', '1034302E'),
    'VIGUD': ('011328N', '1035730E'),
    'VIMAL': ('010942N', '1042353E'),
    'VIRET': ('003940N', '1043511E'),
    'VMR'  : ('022318N', '1035218E'),
    'VOVOS': ('011123N', '1032651E'),
    'VTK'  : ('012455N', '1040120E'),
    'TEKONG' : ('012455N', '1040120E')
}

STARs = {
    'ARAMA1A': ['ARAMA', 'BOBAG', 'BOKIP', 'SAMKO'],
    'ARAMA1B': ['ARAMA', 'BOBAG', 'SAMKO', 'BITAM', 'DOVAN', 'BIPOP'],
    'ASUNA2A': ['ASUNA', 'VAMPO', 'IBASU', 'VEXEL', 'SAMKO'],
    'ASUNA2B': ['ASUNA', 'VAMPO', 'IBASU', 'VEXEL', 'ABVIP', 'AGROT', 'BITAM', 'DOVAN', 'BIPOP'],
    'ELALO1A': ['ELALO', 'KANLA', 'KILOT', 'PIBAP', 'PASPU', 'NYLON', 'POSUB', 'SANAT'],
    'ELALO1B': ['ELALO', 'KANLA', 'KILOT', 'PIBAP', 'PASPU', 'NYLON'],
    'KARTO2A': ['TOMAN', 'KARTO', 'GUNUD', 'KEXAS', 'VIMAL', 'IGNON', 'SANAT'],
    'KARTO2B': ['TOMAN', 'KARTO', 'GUNUD', 'KEXAS', 'LAVAX', 'DOVAN', 'BIPOP'],
    'LEBAR2A': ['PASPU', 'PU', 'SINJON', 'PALGA', 'PAMSI', 'SAMKO'],
    'LEBAR3B': ['REMES', 'SINJON', 'PU', 'BETBA', 'BIDUS'],
    'LELIB3B': ['ARAMA', 'LELIB', 'GUMPU', 'ALFA', 'BIDUS'],
    'MABAL2A': ['MABAL', 'KILOT', 'PIBAP', 'PASPU', 'NYLON', 'POSUB', 'SANAT'],
    'MABAL2B': ['MABAL', 'KILOT', 'PIBAP', 'PASPU', 'NYLON'],
    'REPOV2A': ['REPOV', 'REMES', 'DUMUP', 'SAMKO'],
    'REPOV2B': ['REPOV', 'REMES', 'BITAM', 'DOVAN', 'BIPOP'],
    'TEBUN1A': ['TEBUN', 'VAMPO', 'IBASU', 'VEXEL', 'SAMKO'],
    'TEBUN1B': ['TEBUN', 'VAMPO', 'IBASU', 'VEXEL', 'ABVIP', 'AGROT', 'BITAM', 'DOVAN', 'BIPOP'],
    'UGEBO1A': ['UGEBO', 'GUNUD', 'KEXAS', 'VIMAL', 'IGNON', 'SANAT'],
    'UGEBO1B': ['UGEBO', 'GUNUD', 'KEXAS', 'LAVAX', 'DOVAN', 'BIPOP'],
}

STARs_rwy = {
    'ARAMA1A': 'RWY 02L/02C/02R',
    'ARAMA1B': 'RWY 20R/20C/20L',
    'ASUNA2A': 'RWY 02L/02C/02R',
    'ASUNA2B': 'RWY 20R/20C/20L',
    'ELALO1A': 'RWY 02L/02C/02R',
    'ELALO1B': 'RWY 20R/20C/20L',
    'KARTO2A': 'RWY 02L/02C/02R',
    'KARTO2B': 'RWY 20R/20C/20L',
    'LEBAR2A': 'RWY 02L/02C/02R',
    'LEBAR3B': 'RWY 20R/20C/20L',
    'LELIB3B': 'RWY 20R/20C/20L',
    'MABAL2A': 'RWY 02L/02C/02R',
    'MABAL2B': 'RWY 20R/20C/20L',
    'REPOV2A': 'RWY 02L/02C/02R',
    'REPOV2B': 'RWY 20R/20C/20L',
    'TEBUN1A': 'RWY 02L/02C/02R',
    'TEBUN1B': 'RWY 20R/20C/20L',
    'UGEBO1A': 'RWY 02L/02C/02R',
    'UGEBO1B': 'RWY 20R/20C/20L',
}

SIDs = {
    'ANITO7A': ['MOXIB','EMRIX','HOSBA','VANBU', 'VIRET', 'GURES', 'IKIRO', 'ANITO'],
    'ANITO8B': ['IBIXU', 'IBIVA','ISNOM','ASOMI','UPTEL', 'IDKIV','GIXEM','VASTI','VIRET','GURES','IKIRO','ANITO'],
    'ANITO1C': ['HOSBA','VANBU','VIRET','GURES','IKIRO','ANITO'],
    'ANITO1D': ['UKIBO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IKIRO', 'ANITO'],
    'ANITO7E': ['MOLVO', 'EMRIX', 'HOSBA', 'VANBU', 'VIRET', 'GURES', 'IKIRO', 'ANITO'],
    'ANITO8F': ['LEDOX','LETGO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IKIRO', 'ANITO'],
    'AROSO3A': ['MOXIB','AKOMA','AKMET','AROSO'],
    'AROSO5B': ['IBIXU', 'IBIVA', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','AKMET','AROSO'],
    'AROSO1C': ['AKOMA','AKMET','AROSO'],
    'AROSO1D': ['UKIBO','POVEB','ADPON','SALRU','TEKONG','AKOMA','AKMET','AROSO'],
    'AROSO3E': ['MOLVO','ATRUM','AKOMA','AKMET','AROSO'],
    'AROSO5F': ['LEDOX', 'LETGO', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','AKMET','AROSO'],
    'DODSO1A': ['MOXIB','EMRIX', 'HOSBA', 'VEBMA', 'TOMAN', 'DODSO'],
    'DODSO1B': ['IBIXU', 'IBIVA','DUBOT', 'ERVIV','MUMDU','SEBVO', 'GOTGA','VEBMA', 'TOMAN', 'DODSO'],
    'DODSO1C': ['HOSBA', 'VEBMA', 'TOMAN', 'DODSO'],
    'DODSO1D': ['UKIBO','DUBOT', 'ERVIV','MUMDU','SEBVO', 'GOTGA','VEBMA', 'TOMAN', 'DODSO'],
    'DODSO1E': ['MOLVO','EMRIX', 'HOSBA', 'VEBMA', 'TOMAN', 'DODSO'],
    'DODSO1F': ['LEDOX', 'LETGO', 'DUBOT', 'ERVIV','MUMDU','SEBVO', 'GOTGA','VEBMA', 'TOMAN', 'DODSO'],
    'IDBUD1A': ['MOXIB','EMRIX','HOSBA','VANBU','VIRET','GURES','IDBUD'],
    'IDBUD1B': ['IBIXU','IBIVA','ISNOM','ASOMI','UPTEL','IDKIV','GIXEM','VASTI','VIRET','GURES','IDBUD'],
    'IDBUD1C': ['HOSBA','VANBU','VIRET','GURES','IDBUD'],
    'IDBUD1D': ['UKIBO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IDBUD'],
    'IDBUD1E': ['MOLVO', 'EMRIX', 'HOSBA', 'VANBU', 'VIRET', 'GURES', 'IDBUD'],
    'IDBUD1F': ['LEDOX','LETGO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IDBUD'],
    'KIRDA1A': ['MOXIB','EMRIX','HOSBA','VANBU', 'VIRET', 'GURES', 'IKIRO', 'KIRDA'],
    'KIRDA1B': ['IBIXU', 'IBIVA','ISNOM','ASOMI','UPTEL', 'IDKIV','GIXEM','VASTI','VIRET','GURES','IKIRO','KIRDA'],
    'KIRDA1C': ['HOSBA','VANBU','VIRET','GURES','IKIRO','KIRDA'],
    'KIRDA1D': ['UKIBO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IKIRO', 'KIRDA'],
    'KIRDA1E': ['MOLVO', 'EMRIX', 'HOSBA', 'VANBU', 'VIRET', 'GURES', 'IKIRO', 'KIRDA'],
    'KIRDA1F': ['LEDOX','LETGO', 'ISNOM', 'ASOMI', 'UPTEL', 'IDKIV', 'GIXEM', 'VASTI', 'VIRET', 'GURES', 'IKIRO', 'KIRDA'],
    'MASBO3A': ['MOXIB','AKOMA','AGVAR','SABKA','MASBO'],
    'MASBO5B': ['IBIXU', 'IBIVA', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','AGVAR','SABKA','MASBO'],
    'MASBO1C': ['AKOMA','AGVAR','SABKA','MASBO'],
    'MASBO1D': ['UKIBO','POVEB','ADPON','SALRU','TEKONG','AKOMA','AGVAR','SABKA','MASBO'],
    'MASBO3E': ['MOLVO','ATRUM','AKOMA','AGVAR','SABKA','MASBO'],
    'MASBO5F': ['LEDOX', 'LETGO', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','AGVAR','SABKA','MASBO'],
    'VMR6A': ['MOXIB','AKOMA','VMR'],
    'VMR9B': ['IBIXU', 'IBIVA', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','VMR'],
    'VMR1C': ['AKOMA','VMR'],
    'VMR1D': ['UKIBO','POVEB','ADPON','SALRU','TEKONG','AKOMA','VMR'],
    'VMR6E': ['MOLVO','ATRUM','AKOMA','VMR'],
    'VMR9F': ['LEDOX', 'LETGO', 'DUBOT','ADPON','SALRU','TEKONG','AKOMA','VMR'],
    'MIBEL1A': ['MOXIB','EMRIX','HOSBA','VANBU', 'IGOSI','ASITI','VIBOG','ISGIL','BISOV','MIBEL'],
    'MIBEL1B': ['IBIXU', 'IBIVA','SAMKO','VIBOG','ISGIL','BISOV','MIBEL'],
    'MIBEL1C': ['HOSBA','VANBU','IGOSI','ASITI','VIBOG','ISGIL','BISOV','MIBEL'],
    'MIBEL1D': ['UKIBO', 'VIGUD', 'SAMKO','VIBOG','ISGIL','BISOV','MIBEL'],
    'MIBEL1E': ['MOLVO', 'EMRIX', 'HOSBA', 'VANBU', 'IGOSI','ASITI','VIBOG','ISGIL','BISOV','TAROS'],
    'MIBEL1F': ['LEDOX','LETGO', 'SAMKO','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1A': ['MOXIB','EMRIX','HOSBA','VANBU', 'IGOSI','ASITI','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1B': ['IBIXU', 'IBIVA','SAMKO','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1C': ['HOSBA','VANBU','IGOSI','ASITI','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1D': ['UKIBO', 'VIGUD', 'SAMKO','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1E': ['MOLVO', 'EMRIX', 'HOSBA', 'VANBU', 'IGOSI','ASITI','VIBOG','ISGIL','BISOV','TAROS'],
    'TAROS1F': ['LEDOX','LETGO', 'SAMKO','VIBOG','ISGIL','BISOV','TAROS'],
    'TOMAN3A': ['MOXIB','EMRIX','HOSBA','VEBMA','TOMAN'],
    'TOMAN5B': ['IBIXU', 'IBIVA','DUBOT','ERVIV','MUMDU','SEBVO','GOTGA','VEBMA','TOMAN'],
    'TOMAN1C': ['HOSBA','VEBMA','TOMAN'],
    'TOMAN1D': ['UKIBO', 'DUBOT','ERVIV','MUMDU','SEBVO','GOTGA','VEBMA','TOMAN'],
    'TOMAN3E': ['MOLVO', 'EMRIX', 'HOSBA', 'VEBMA','TOMAN'],
    'TOMAN5F': ['LEDOX','LETGO', 'DUBOT','ERVIV','MUMDU','SEBVO','GOTGA','VEBMA','TOMAN'],
    'VOVOS1B': ['IBIXU', 'IBIVA','SAMKO','BOKIP','ATLEX','VOVOS'],
    'VOVOS1D': ['UKIBO', 'VIGUD', 'SAMKO','BOKIP','ATLEX','VOVOS'],
    'VOVOS1F': ['LEDOX','LETGO', 'SAMKO','BOKIP','ATLEX','VOVOS'],
    'CHA1C': [],
    'CHA1D': [],
}

SIDs_rwy = {
    'ANITO7A': 'RWY 02C',
    'ANITO8B': 'RWY 20C',
    'ANITO1C': 'RWY 02R',
    'ANITO1D': 'RWY 20L',
    'ANITO7E': 'RWY 02L',
    'ANITO8F': 'RWY 20R',
    'AROSO3A': 'RWY 02C',
    'AROSO5B': 'RWY 20C',
    'AROSO1C': 'RWY 02R',
    'AROSO1D': 'RWY 20L',
    'AROSO3E': 'RWY 02L',
    'AROSO5F': 'RWY 20R',
    'DODSO1A': 'RWY 02C',
    'DODSO1B': 'RWY 20C',
    'DODSO1C': 'RWY 02R',
    'DODSO1D': 'RWY 20L',
    'DODSO1E': 'RWY 02L',
    'DODSO1F': 'RWY 20R',
    'IDBUD1A': 'RWY 02C',
    'IDBUD1B': 'RWY 20C',
    'IDBUD1C': 'RWY 02R',
    'IDBUD1D': 'RWY 20L',
    'IDBUD1E': 'RWY 02L',
    'IDBUD1F': 'RWY 20R',
    'KIRDA1A': 'RWY 02C',
    'KIRDA1B': 'RWY 20C',
    'KIRDA1C': 'RWY 02R',
    'KIRDA1D': 'RWY 20L',
    'KIRDA1E': 'RWY 02L',
    'KIRDA1F': 'RWY 20R',
    'MASBO3A': 'RWY 02C',
    'MASBO5B': 'RWY 20C',
    'MASBO1C': 'RWY 02R',
    'MASBO1D': 'RWY 20L',
    'MASBO3E': 'RWY 02L',
    'MASBO5F': 'RWY 20R',
    'VMR6A': 'RWY 02C',
    'VMR9B': 'RWY 20C',
    'VMR1C': 'RWY 02R',
    'VMR1D': 'RWY 20L',
    'VMR6E': 'RWY 02L',
    'VMR9F': 'RWY 20R',
    'MIBEL1A': 'RWY 02C',
    'MIBEL1B': 'RWY 20C',
    'MIBEL1C': 'RWY 02R',
    'MIBEL1D': 'RWY 20L',
    'MIBEL1E': 'RWY 02L',
    'MIBEL1F': 'RWY 20R',
    'TAROS1A': 'RWY 02C',
    'TAROS1B': 'RWY 20C',
    'TAROS1C': 'RWY 02R',
    'TAROS1D': 'RWY 20L',
    'TAROS1E': 'RWY 02L',
    'TAROS1F': 'RWY 20R',
    'TOMAN3A': 'RWY 02C',
    'TOMAN5B': 'RWY 20C',
    'TOMAN1C': 'RWY 02R',
    'TOMAN1D': 'RWY 20L',
    'TOMAN3E': 'RWY 02L',
    'TOMAN5F': 'RWY 20R',
    'VOVOS1B': 'RWY 20C',
    'VOVOS1D': 'RWY 20L',
    'VOVOS1F': 'RWY 20R',
    'CHA1C': 'RWY 02R',
    'CHA1D': 'RWY 20L',
}

holding_pts_app = ['BOBAG', 'HOSBA', 'KEXAS','LAMA', 'NYLON','REMES','SAMKO','SINJON','TUSPI','VAMPO']
holding_pts_acc = ['ELALO','KARTO','KILOT','MABAL','REPOV','UGEBO'] 


In [7]:
len(waypoints_all.keys()), len(STAR_SID_waypoints.keys())

(183, 107)

In [8]:
combined_waypoints = waypoints_all | STAR_SID_waypoints
len(combined_waypoints.keys())

228

In [27]:
# SINGAPORE FIR SECTORS
SG_FIR_sector1_coordinates = [
    ('004237N', '103523E'),
    ('003513N', '103540E'),
    ('001617N', '103611E'),
    ('000000N', '104432E'),
    ('000000N', '105034E'),
    ('001030N', '104605E'),
    ('003534N', '104414E'),
    ('001955N', '105110E'),
    ('013228N', '104239E'),
    ('013112N', '103633E'),
    ('013013N', '103502E'),
    ('012707N', '103321E'),
    ('010711N', '103511E'),
    ('011543N', '103212E'),
    ('004237N', '103523E')  # closed polygon
]

SG_FIR_sector2_coordinates = [
    ('020008N', '1034828E'), 
    ('013014N', '1035025E'), 
    ('012707N', '1032829E'), 
    ('011543N', '1032139E'), 
    ('010711N', '1034717E'), 
    ('004238N', '1035244E'), 
    ('003513N', '1035407E'), 
    ('001617S', '1035719E'), 
    ('013429N', '1022353E'), 
    ('012204N', '1030206E'), 
    ('014517N', '1031303E'), 
    ('014215N', '1031717E'),
    ('020008N', '1034828E')
]

SG_FIR_sector3_coordinates = [
    ('030725N', '1034442E'), 
    ('030735N', '1041256E'), 
    ('023558N', '1044459E'), 
    ('013228N', '1042406E'), 
    ('013118N', '1040128E'), 
    ('013112N', '1035936E'), 
    ('013014N', '1035025E'), 
    ('020008N', '1034828E'), 
    ('022203N', '1034714E'), 
    ('025220N', '1033324E'), 
    ('025418N', '1034606E'),
    ('030725N', '1034442E')
]

SG_FIR_sector4_coordinates = [
    ('023721N', '1044337E'), 
    ('023935N', '1045028E'), 
    ('023000N', '1053000E'), 
    ('023333N', '1054816E'), 
    ('012500N', '1061648E'), 
    ('010831N', '1055908E'), 
    ('001956N', '1050703E'), 
    ('003534N', '1044152E'), 
    ('013228N', '1042406E'), 
    ('023558N', '1044459E'),
    ('023721N', '1044337E')
]

SG_FIR_sector5_coordinates = [
    ('035421N', '1063418E'), 
    ('074159N', '1091159E'), 
    ('103000N', '1140000E'), 
    ('082500N', '1163000E'), 
    ('025050N', '1091629E'), 
    ('045700N', '1081619E'), 
    ('050012N', '1080132E'), 
    ('045904N', '1075525E'), 
    ('045203N', '1074625E'), 
    ('043820N', '1073315E'), 
    ('041312N', '1071743E'), 
    ('035237N', '1063555E'),
    ('035421N', '1063418E')
]

SG_FIR_sector6_coordinates = [
    ('033600N', '1042208E'), 
    ('040311N', '1054617E'), 
    ('041717N', '1061247E'), 
    ('035237N', '1063555E'), 
    ('034417N', '1061859E'), 
    ('033022N', '1055053E'), 
    ('031727N', '1052959E'), 
    ('023331N', '1054816E'), 
    ('023001N', '1052958E'), 
    ('023935N', '1045028E'), 
    ('023721N', '1044337E'), 
    ('030735N', '1041256E'),
    ('033600N', '1042208E')
]

SG_FIR_sector7_coordinates = [
    ('045000N', '1034400E'), 
    ('045223N', '1041441E'), 
    ('044841N', '1052247E'), 
    ('041716N', '1061247E'), 
    ('040311N', '1054617E'), 
    ('033600N', '1042208E'), 
    ('030735N', '1041256E'), 
    ('030725N', '1034442E'), 
    ('034000N', '1034000E'),
    ('045000N', '1034400E')
]

SG_FIR_sector8_coordinates = [
    ('064459N', '1024001E'), 
    ('065955N', '1025953E'), 
    ('070000N', '1080000E'), 
    ('074159N', '1091159E'), 
    ('035421N', '1063418E'), 
    ('041717N', '1061246E'), 
    ('044841N', '1052247E'), 
    ('045223N', '1041441E'), 
    ('045000N', '1034400E'),
    ('064459N', '1024001E')
]


In [8]:
for star_name, star_waypoints in STARs.items():
    route = []
    for waypoints in star_waypoints:
        wpt_coordinates = (STAR_SID_waypoints[waypoints])
        route.append(wpt_coordinates)
    star_lat, star_lon = parse_coordinates(route)
    print(star_lat, '\n', star_lon)
    print("\n")
    # star_lat, star_lon = parse_coordinates(route)

[1.6, 1.03333, 1.06667, 1.08333] 
 [103.11667, 103.48333, 103.71667, 103.86667]


[1.6, 1.03333, 1.08333, 1.13333, 1.31667, 1.51667] 
 [103.11667, 103.48333, 103.86667, 104.11667, 104.2, 104.16667]


[0.98333, 0.96667, 0.95, 0.98333, 1.08333] 
 [103.15, 103.41667, 103.56667, 103.7, 103.86667]


[0.98333, 0.96667, 0.95, 0.98333, 1.0, 1.01667, 1.13333, 1.31667, 1.51667] 
 [103.15, 103.41667, 103.56667, 103.7, 103.83333, 103.96667, 104.11667, 104.2, 104.16667]


[4.2, 3.75, 3.03333, 2.5, 1.98333, 1.6, 1.45, 1.11667] 
 [104.55, 104.6, 104.66667, 104.1, 104.1, 104.1, 104.11667, 103.98333]


[4.2, 3.75, 3.03333, 2.5, 1.98333, 1.6] 
 [104.55, 104.6, 104.66667, 104.1, 104.1, 104.1]


[1.35, 1.18333, 1.16667, 1.16667, 1.15, 1.13333, 1.11667] 
 [105.78333, 105.55, 105.1, 104.8, 104.38333, 104.2, 103.98333]


[1.35, 1.18333, 1.16667, 1.16667, 1.15, 1.31667, 1.51667] 
 [105.78333, 105.55, 105.1, 104.8, 104.45, 104.2, 104.16667]


[1.98333, 1.41667, 1.21667, 1.16667, 1.06667, 1.08333] 
 [104.1, 103

In [9]:
route

[('003813N', '1052432E'),
 ('011042N', '1050618E'),
 ('011019N', '1044818E'),
 ('010950N', '1042714E'),
 ('011938N', '1041249E'),
 ('013122N', '1041018E')]

In [10]:
star_lat, star_lon = parse_coordinates(route)
star_lat, star_lon

([0.63333, 1.16667, 1.16667, 1.15, 1.31667, 1.51667],
 [105.4, 105.1, 104.8, 104.45, 104.2, 104.16667])

### Aerodromes

#### WSSS - Singapore Changi Airport

In [30]:
WSSS_lat, WSSS_lon = round_latlon(1.359189613098253, 103.98934153635464)

# RWY 02L/20R
WSSS_02L = [
   (round_latlon(1.348860502342621, 103.97769146161956)),
   (round_latlon(1.349054807582341, 103.97720033915476)),
   (round_latlon(1.376217458893540, 103.98867930690206)),
   (round_latlon(1.376008977292466, 103.98916478672038)),
   (round_latlon(1.348860502342621, 103.97769146161956))
]


lat_list_WSSS_02L, lon_list_WSSS_02L = create_latlon_list(WSSS_02L)

#  RWY 02C/20C
WSSS_02C = [
   (round_latlon(1.3287046741333952, 103.98521622453343)),
   (round_latlon(1.328872267112968, 103.98472135698383)),
   (round_latlon(1.3621357324395549, 103.99876547895391)),
   (round_latlon(1.3619269999140773, 103.99924531754634)),
   (round_latlon(1.3287046741333952, 103.98521622453343))
]

lat_list_WSSS_02C, lon_list_WSSS_02C = create_latlon_list(WSSS_02C)

# RWY 02R/20L
WSSS_02R = [
   (round_latlon(1.3224891126717109, 103.99960374943434)),
   (round_latlon(1.3222734166294323, 104.00008783502287)),
   (round_latlon(1.3555766292118474, 104.01415695079532)),
   (round_latlon(1.3557861428974194, 104.01367190285114)),
   (round_latlon(1.3224891126717109, 103.99960374943434))
]

lat_list_WSSS_02R, lon_list_WSSS_02R = create_latlon_list(WSSS_02R)

#### WSSL - Seletar Airport

In [12]:
WSSL_lat, WSSL_lon = round_latlon(1.4136967080965424, 103.86904111139437)

# RWY 03/21
WSSL_03 = [
   (round_latlon(1.408673537176929, 103.86200363994745)),
   (round_latlon(1.408456703041756, 103.86233855856294)),
   (round_latlon(1.422326363854819, 103.87140043318387)),
   (round_latlon(1.4225502711432483, 103.87106198010089)),
   (round_latlon(1.408673537176929, 103.86200363994745))
]

lat_list_WSSL_03, lon_list_WSSL_03 = create_latlon_list(WSSL_03)

#### WSAP - Paya Lebar Air Base

In [13]:
WSAP_lat, WSAP_lon = round_latlon(1.352094883242226, 103.90152577664577)

# RWY 02/20
WSAP_02 = [
    (round_latlon(1.3447803949278216, 103.90256530140103)),
    (round_latlon(1.3445732371900398, 103.9030578350208)),
    (round_latlon(1.3760655064105025, 103.91644498588705)),
    (round_latlon(1.3762680078768263, 103.9159719869955)),
    (round_latlon(1.3447803949278216, 103.90256530140103))
]

lat_list_WSAP_02, lon_list_WSAP_02 = create_latlon_list(WSAP_02)

#### WSAT - Tengah Air Base

In [14]:
WSAT_lat, WSAT_lon = round_latlon(1.385462843457273, 103.71144740297487)

# RWY 18/36
WSAT_18 = [
    (round_latlon(1.3752731999645278, 103.70697211283645)),
    (round_latlon(1.375235980861987, 103.70737069097719)),
    (round_latlon(1.399991571917424, 103.70951889541763)),
    (round_latlon(1.400025205884199, 103.70911920458241)),
    (round_latlon(1.3752731999645278, 103.70697211283645))
]

lat_list_WSAT_18, lon_list_WSAT_18 = create_latlon_list(WSAT_18)

# RWY 18/36
WSAT_36 = [
    (round_latlon(1.386800170550625, 103.69848430518792)),
    (round_latlon(1.3867508931521588, 103.69874660816976)),
    (round_latlon(1.3973437130401112, 103.70054235799107)),
    (round_latlon(1.3973913062258823, 103.70027956545623)),
    (round_latlon(1.386800170550625, 103.69848430518792))
]
lat_list_WSAT_36, lon_list_WSAT_36 = create_latlon_list(WSAT_36)

#### WSAG - Sembawang Air Base

In [ ]:
WSAG_lat, WSAG_lon = round_latlon(1.4250523966876878, 103.82021133744765)

# RWY 04/22
WSAG_04 = [
    (round_latlon(1.418568399607689, 103.80439520306064)),
    (round_latlon(1.4182792857561743, 103.8047032154432)),
    (round_latlon(1.4342411191617723, 103.81932911048153)),
    (round_latlon(1.4345202460125308, 103.81902294494982)),
    (round_latlon(1.418568399607689, 103.80439520306064))
]

lat_list_WSAG_04, lon_list_WSAG_04 = create_latlon_list(WSAG_04)

# RWY H05/H23 - Helicopter
WSAG_H05 = [
    (round_latlon(1.42378188191484, 103.81296499850109)),
    (round_latlon(1.4235935079284219, 103.8131424004795)),
    (round_latlon(1.4292660593821376, 103.81909895043836)),
    (round_latlon(1.4294538539377502, 103.81892241387786)),
    (round_latlon(1.42378188191484, 103.81296499850109))
]
lat_list_WSAG_H05, lon_list_WSAG_H05 = create_latlon_list(WSAG_H05)

#### Pulau Sudong Military Airstrip

In [16]:
sudong_lat, sudong_lon = round_latlon(1.2053925669851036, 103.71955295659919)

# RWY 09/27
sudong_09 = [
    (round_latlon(1.2056387286415973, 103.70991142954433)),
    (round_latlon(1.2052480172095386, 103.70991537699803)),
    (round_latlon(1.2052517304702854, 103.72862712516758)),
    (round_latlon(1.2056462253345117, 103.7286271713864)),
    (round_latlon(1.2056387286415973, 103.70991142954433))
]

lat_list_sudong_09, lon_list_sudong_09 = create_latlon_list(sudong_09)

#### WMKJ - Senai Intl Airport

In [17]:
WMKJ_lat, WMKJ_lon = round_latlon(1.6356196100740554, 103.66699990520668)

# RWY 16/34
WMKJ_16 = [
    (round_latlon(1.6231851849288097, 103.67554291541757)),
    (round_latlon(1.6233170292876669, 103.67591561601094)),
    (round_latlon(1.655579824575071, 103.66416903882948)),
    (round_latlon(1.6554421729698645, 103.66379838867114)),
    (round_latlon(1.6231851849288097, 103.67554291541757))
]

lat_list_WMKJ_16, lon_list_WMKJ_16 = create_latlon_list(WMKJ_16)

#### WIDD - Hang Nadim Intl Airport

In [18]:
WIDD_lat, WIDD_lon = round_latlon(1.1201865768779822, 104.11395990919655)

# RWY 04/22
WIDD_04 = [
    (round_latlon(1.1074811935602336, 104.10664452214822)),
    (round_latlon(1.1072214951625803, 104.10693869334261)),
    (round_latlon(1.1344260137807463, 104.1309419488069)),
    (round_latlon(1.1346869751031672, 104.13064648796646)),
    (round_latlon(1.1074811935602336, 104.10664452214822))
]
lat_list_WIDD_04, lon_list_WIDD_04 = create_latlon_list(WIDD_04)

#### WIDN - Raja Haji Fisabilillah Intl Airport 

In [19]:
WIDN_lat, WIDN_lon = round_latlon(0.9238491007025411, 104.53015244650595)

# RWY 04/22
WIDN_04 = [
    (round_latlon(0.9167554692823954, 104.52644320947208)),
    (round_latlon(0.9164849781439796, 104.52672786189764)),
    (round_latlon(0.9300551317336074, 104.53927669805813)),
    (round_latlon(0.9303215504911958, 104.53899093593742)),
    (round_latlon(0.9167554692823954, 104.52644320947208))
]

lat_list_WIDN_04, lon_list_WIDN_04 = create_latlon_list(WIDN_04)

#### WIDT - Raja Haji Abdullah Airport

In [20]:
WIDT_lat, WIDT_lon = round_latlon(1.0514440421356532, 103.39121781470347)

# RWY 09/27
WIDT_09 = [
    (round_latlon(1.0533824959702007, 103.3848862135349)),
    (round_latlon(1.0531307010683926, 103.38487130210696)),
    (round_latlon(1.0523973087254361, 103.39777448320025)),
    (round_latlon(1.052652736672405, 103.39778725675268)),
    (round_latlon(1.0533824959702007, 103.3848862135349))
]

lat_list_WIDT_09, lon_list_WIDT_09 = create_latlon_list(WIDT_09)

### Interactive Map Plot

In [28]:
# Plot Map
fig = go.Figure()
version = '1.0.1'

#region RWY customisations
rwy_width = 1.5
airport_marker_size = 12
color_airport = 'dodgerblue'
color_rwy = 'dodgerblue'
color_rwy_fill = 'rgba(0, 0, 255, 0.1)'

# Inactive/Restricted runways
color_rwy_inactive = 'red'          
color_rwy_inactive_fill = 'rgba(255, 0, 0, 0.7)'
#endregion

########## ------------------------------- ##########

#region Singapore
#region WSSS

# Location marker
fig.add_trace(go.Scattermap(
    lat=[WSSS_lat],
    lon=[WSSS_lon],
    mode='markers+text',
    marker=dict(size=airport_marker_size, color=color_airport, symbol= 'circle'),
    # text='WSSS',
    # textfont=dict(weight="bold"),
    textposition='top right',
    name='Singapore Changi Airport (WSSS)',
    hovertext='WSSS',
    legendgroup='changi',
    legendgrouptitle_text='Singapore Changi Airport',
    showlegend=True,
    hovertemplate='📍 %{fullData.name}<extra></extra>'
))

# RWY 02L/20R

fig.add_trace(go.Scattermap(
    lat=lat_list_WSSS_02L,
    lon=lon_list_WSSS_02L,
    mode='lines',
    fill='toself',
    fillcolor=color_rwy_fill,
    line=dict(color=color_rwy, width= rwy_width),
    name='RWY 02L/20R',
    legendgroup='changi'
))

#  RWY 02C/20C

fig.add_trace(go.Scattermap(
    lat=lat_list_WSSS_02C,
    lon=lon_list_WSSS_02C,
    mode='lines',
    fill='toself',
    fillcolor=color_rwy_fill,
    line=dict(color=color_rwy, width= rwy_width),
    name='RWY 02C/20C',
    legendgroup='changi'
))

# RWY 02R/20L

fig.add_trace(go.Scattermap(
    lat=lat_list_WSSS_02R,
    lon=lon_list_WSSS_02R,
    mode='lines',
    fill='toself',
    fillcolor=color_rwy_inactive_fill,
    line=dict(color=color_rwy_inactive, width= rwy_width),
    name='RWY 02R/20L',
    legendgroup='changi'
))



#endregion

#region WSSL

# Location marker
fig.add_trace(go.Scattermap(
    lat=[WSSL_lat],
    lon=[WSSL_lon],
    mode='markers+text',
    marker=dict(size=airport_marker_size, color=color_airport, symbol= 'circle'),
    textposition='top right',
    name='Seletar Airport (WSSL)',
    hovertext='WSSL',
    legendgroup='seletar',
    legendgrouptitle_text='Seletar Airport',
    showlegend=True,
    hovertemplate='📍 %{fullData.name}<extra></extra>'
))

# RWY 03/21

fig.add_trace(go.Scattermap(
    lat=lat_list_WSSL_03,
    lon=lon_list_WSSL_03,
    mode='lines',
    fill='toself',
    fillcolor=color_rwy_fill,
    line=dict(color=color_rwy, width= rwy_width),
    name='RWY 03/21',
    legendgroup='seletar'
))
#endregion

#region WSAP

add_airport(
    fig=fig,
    name='Paya Lebar Air Base',
    code='WSAP',
    lat=WSAP_lat,
    lon=WSAP_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='payalebar',
    showlegend=True
)

# RWY 02/20
add_runway(
    fig=fig,
    name='RWY 02/20',
    lat_list=lat_list_WSAP_02,
    lon_list=lon_list_WSAP_02,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='payalebar',
    showlegend=True
)
#endregion

#region WSAT

add_airport(
    fig=fig,
    name='Tengah Air Base',
    code='WSAT',
    lat=WSAT_lat,
    lon=WSAT_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='tengah',
    showlegend=True
)

# RWY 18/36
add_runway(
    fig=fig,
    name='RWY 18/36',
    lat_list=lat_list_WSAT_18,
    lon_list=lon_list_WSAT_18,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='tengah',
    showlegend=True
)

# RWY 18/36 second
add_runway(
    fig=fig,
    name='Military Airstrip',
    lat_list=lat_list_WSAT_36,
    lon_list=lon_list_WSAT_36,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='tengah',
    showlegend=True
)

#endregion

#region WSAG
add_airport(
    fig=fig,
    name='Sembawang Air Base',
    code='WSAG',
    lat=WSAG_lat,
    lon=WSAG_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='sembawang',
    showlegend=True
)

# RWY 04/22
add_runway(
    fig=fig,
    name='RWY 04/22',
    lat_list=lat_list_WSAG_04,
    lon_list=lon_list_WSAG_04,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='sembawang',
    showlegend=True
)
# RWY H05/H23 - Helicopter
add_runway(
    fig=fig,
    name='RWY H05/H23',
    lat_list=lat_list_WSAG_H05,
    lon_list=lon_list_WSAG_H05,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='sembawang',
    showlegend=True
)
#endregion

#region Sudong

add_runway(
    fig=fig,
    name='RWY 09/27',
    lat_list=lat_list_sudong_09,
    lon_list=lon_list_sudong_09,
    fillcolor=color_rwy_inactive_fill,
    linecolor=color_rwy_inactive,
    linewidth=rwy_width,
    legendgroup='sudong',
    legendgrouptitle_text = 'Pulau Sudong Military Airstrip',
    showlegend=True
)
#endregion

#endregion

########## ------------------------------- ##########

#region Malaysia

#region WMKJ
add_airport(
    fig=fig,
    name='Senai Intl Airport',
    code='WMKJ',
    lat=WMKJ_lat,
    lon=WMKJ_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='senai',
    showlegend=True
)
# RWY 16/34
add_runway(
    fig=fig,
    name='RWY 16/34',
    lat_list=lat_list_WMKJ_16,
    lon_list=lon_list_WMKJ_16,
    fillcolor=color_rwy_fill,
    linecolor=color_rwy,
    linewidth=rwy_width,
    legendgroup='senai',
    showlegend=True
)
#endregion

#endregion

########## ------------------------------- ##########

#region Indonesia

#region WIDD
add_airport(
    fig=fig,
    name='Hang Nadim Intl Airport',
    code='WIDD',
    lat=WIDD_lat,
    lon=WIDD_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='hangnadim',
    showlegend=True
)
# RWY 04/22
add_runway(
    fig=fig,
    name='RWY 04/22',
    lat_list=lat_list_WIDD_04,
    lon_list=lon_list_WIDD_04,
    fillcolor=color_rwy_fill,
    linecolor=color_rwy,
    linewidth=rwy_width,
    legendgroup='hangnadim',
    showlegend=True
)
#endregion

#region WIDN
add_airport(
    fig=fig,
    name='Raja Haji Fisabilillah Intl Airport',
    code='WIDN',
    lat=WIDN_lat,
    lon=WIDN_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='tanjungpinang',
    showlegend=True
)
# RWY 04/22
add_runway(
    fig=fig,
    name='RWY 04/22',
    lat_list=lat_list_WIDN_04,
    lon_list=lon_list_WIDN_04,
    fillcolor=color_rwy_fill,
    linecolor=color_rwy,
    linewidth=rwy_width,
    legendgroup='tanjungpinang',
    showlegend=True
)
#endregion

#region WIDT
add_airport(
    fig=fig,
    name='Raja Haji Abdullah Airport',
    code='WIDT',
    lat=WIDT_lat,
    lon=WIDT_lon,
    color=color_airport,
    size=airport_marker_size,
    legendgroup='karimun',
    showlegend=True
)
# RWY 09/27
add_runway(
    fig=fig,
    name='RWY 09/27',
    lat_list=lat_list_WIDT_09,
    lon_list=lon_list_WIDT_09,
    fillcolor=color_rwy_fill,
    linecolor=color_rwy,
    linewidth=rwy_width,
    legendgroup='karimun',
    showlegend=True
)
#endregion

#endregion

########## ------------------------------- ##########

#region FIR boundaries

#region Singapore FIR sectors

# Sector 1
add_sector(
    fig=fig,
    name='Sector 1',
    coordinates=SG_FIR_sector1_coordinates,
    fillcolor='mediumslateblue',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    legendgrouptitle_text='Singapore FIR Sectors',
    legendrank=2,
    showlegend=True
)

# Sector 2
add_sector(
    fig=fig,
    name='Sector 2',
    coordinates=SG_FIR_sector2_coordinates,
    fillcolor='aqua',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 3
add_sector(
    fig=fig,
    name='Sector 3',
    coordinates=SG_FIR_sector3_coordinates,
    fillcolor='violet',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 4
add_sector(
    fig=fig,
    name='Sector 4',
    coordinates=SG_FIR_sector4_coordinates,
    fillcolor='yellow',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 5
add_sector(
    fig=fig,
    name='Sector 5',
    coordinates=SG_FIR_sector5_coordinates,
    fillcolor='cornflowerblue',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 6
add_sector(
    fig=fig,
    name='Sector 6',
    coordinates=SG_FIR_sector6_coordinates,
    fillcolor='lightgreen',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 7
add_sector(
    fig=fig,
    name='Sector 7',
    coordinates=SG_FIR_sector7_coordinates,
    fillcolor='lightskyblue',
    linecolor='grey',
    opacity=0.3,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

# Sector 8
add_sector(
    fig=fig,
    name='Sector 8',
    coordinates=SG_FIR_sector8_coordinates,
    fillcolor='navajowhite',
    linecolor='grey',
    opacity=0.4,
    linewidth=1,
    legendgroup='SG FIR',
    showlegend=True
)

#endregion

#region JAKARTA_FIR_delegated_coordinates
jkt_fir_lat, jkt_fir_long = parse_coordinates(JAKARTA_FIR_delegated_coordinates)

fig.add_trace(go.Scattermap(
    lat=jkt_fir_lat,
    lon=jkt_fir_long,
    mode='lines+markers',
    fill='none',  # fill the polygon
    fillcolor='rgba(255, 0, 0, 0.2)',  # translucent red
    line=dict(color='grey', width=0.7),
    name='JAKARTA FIR',
    legendgroup="FIR",
    showlegend=True
))
#endregion

#region KL_FIR_vested1_coordinates
kl_lat_1, kl_lon_1 = parse_coordinates(KL_FIR_vested1_coordinates)
kl_lat_2, kl_lon_2 = parse_coordinates(KL_FIR_vested2_coordinates)

fig.add_trace(go.Scattermap(
    lat=kl_lat_1,
    lon=kl_lon_1,
    mode='lines+markers',
    fill='none',  # fill the polygon
    fillcolor='rgba(255, 0, 0, 0.2)',  # translucent red
    line=dict(color='grey', width=1),
    name='KUALA LUMPUR FIR 1',
    legendgroup="FIR",
    showlegend=True
))

fig.add_trace(go.Scattermap(
    lat=kl_lat_2,
    lon=kl_lon_2,
    mode='lines+markers',
    fill='none',  # fill the polygon
    fillcolor='rgba(255, 0, 0, 0.2)',  # translucent red
    line=dict(color='grey', width=1),
    name='KUALA LUMPUR FIR 2',
    legendgroup="FIR",
    showlegend=True
))
#endregion

#region Singapore FIR
sg_fir_lat, sg_fir_long = parse_coordinates(SG_FIR_coordinates)

fig.add_trace(go.Scattermap(
    lat=sg_fir_lat,
    lon=sg_fir_long,
    mode='lines+markers',
    fill='none',  # fill the polygon
    fillcolor='rgba(255, 0, 0, 0.2)',  # translucent red
    line=dict(color='black', width=1.5),
    name='SINGAPORE FIR',
    legendgroup="FIR",
    legendrank= 1,
    legendgrouptitle_text="FIR",
    showlegend=True
))
#region fig.add_trace(go.Scattermap(
#     lat=[5],
#     lon=[5],
#     mode='markers+text',
#     marker=dict(size=1000, color='rgba(0,0,0,0)'),  # invisible marker
#     text=['Restricted Area A'],
#     textposition='middle center',
#     textfont=dict(size=1000, color='black', family='Arial'),
#     showlegend=True
# ))
#endregion
#endregion

#endregion

########## ------------------------------- ##########

#region Waypoints
for waypoint_name, waypoint_coordinates in waypoints_all.items():
    waypoint_lat, waypoint_lon = parse_coordinates([waypoint_coordinates])
    fig.add_trace(go.Scattermap(
        lat=waypoint_lat,
        lon=waypoint_lon,
        mode='markers+text',
        marker=dict(size=5, color='royalblue'),
        # text=[waypoint_name],
        textposition='top right',
        name=waypoint_name,
        legendgroup="Waypoints",
        legendgrouptitle_text="Waypoints",
        showlegend=False
    ))
#endregion

########## ------------------------------- ##########

#region STARs
for star_name, star_waypoints in STARs.items():
    route = []
    for waypoints in star_waypoints:
        wpt_coordinates = (STAR_SID_waypoints[waypoints])
        route.append(wpt_coordinates)
    star_lat, star_lon = parse_coordinates(route)
    fig.add_trace(go.Scattermap(
        lat=star_lat,
        lon=star_lon,
        mode='lines+markers',
        line=dict(color='salmon', width=1),
        hovertext=star_waypoints,
        name=f'{star_name} : {STARs_rwy[star_name]}',
        # legendgroup="STAR",
        legendgrouptitle_text="STAR",
        legendgrouptitle_font=dict(color='salmon'),
        showlegend=True
    ))
#endregion

########## ------------------------------- ##########

#region SIDs
for sid_name, sid_waypoints in SIDs.items():
    route = []
    for waypoints in sid_waypoints:
        wpt_coordinates = (STAR_SID_waypoints[waypoints])
        route.append(wpt_coordinates)
    sid_lat, sid_lon = parse_coordinates(route)
    fig.add_trace(go.Scattermap(
        lat=sid_lat,
        lon=sid_lon,
        mode='lines+markers',
        line=dict(color='mediumseagreen', width=1),
        hovertext=sid_waypoints,
        name=f'{sid_name} : {SIDs_rwy[sid_name]}',
        # legendgroup="SID",
        legendgrouptitle_text="SID",
        legendgrouptitle_font=dict(color='mediumseagreen'),
        showlegend=True
    ))
#endregion

########## ------------------------------- ##########

#region Figure layout
fig.update_layout(
    map=dict(
        style='carto-positron',     # 'white-bg' 'stamen-watercolor'
        center=dict(lat=WSSS_lat+3, lon=WSSS_lon+3),
        zoom=5.5
    ),
    width=1600,
    height=1080,
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    legend=dict(
        title='Legend',
        x=1.0,
        y=0.99,
        bgcolor='white',
        bordercolor='black',
        borderwidth=0,
        
    ))
# Map Title

# fig.update_layout(
#     title=dict(
#         text=f'version: {version}',
#         x=0.02,
#         y=0.01,
#         xanchor='center',
#         font=dict(size=10, color='black', family="Raleway"),
#     )
# )

fig.add_trace(go.Scattermap(                # Add version info in legend
    lat=[None],  # No actual point
    lon=[None],
    mode='lines',
    line=dict(color='black', width=2),
    name=f'🗺️ map version: {version}',
    showlegend=True,
    hoverinfo='skip',
    legendgroup='meta'
))
# fig.update_layout(legend=dict(groupclick='toggleitem'))
#endregion
fig.show()


In [29]:
runway_procedures = {
    "RWY 02L": {
        "STARs":["ARAMA1A", "ASUNA2A", "ELALO1A", "KARTO2A", "LEBAR2A", "MABAL2A", "REPOV2A", "TEBUN1A", "UGEBO1A"],
        "SIDs": ["ANITO7E", "AROSO3E", "DODSO1E", "IDBUD1E", "KIRDA1E", "MASBO3E", "VMR6E", "MIBEL1E", "TAROS1E", "TOMAN3E"]
    },
    "RWY 02C": {
        "STARs":["ARAMA1A", "ASUNA2A", "ELALO1A", "KARTO2A", "LEBAR2A", "MABAL2A", "REPOV2A", "TEBUN1A", "UGEBO1A"],
        "SIDs": ["ANITO7A", "AROSO3A", "DODSO1A", "IDBUD1A", "KIRDA1A", "MASBO3A", "VMR6A", "MIBEL1A", "TAROS1A", "TOMAN3A"]
    },
    "RWY 02R": {
        "STARs":["ARAMA1A", "ASUNA2A", "ELALO1A", "KARTO2A", "LEBAR2A", "MABAL2A", "REPOV2A", "TEBUN1A", "UGEBO1A"],
        "SIDs": ["ANITO1C", "AROSO1C", "DODSO1C", "IDBUD1C", "KIRDA1C", "MASBO1C", "VMR1C", "MIBEL1C", "TAROS1C", "TOMAN1C"]
    },
    "RWY 20R": {
        "STARs":["ARAMA1B", "ASUNA2B", "ELALO1B", "KARTO2B", "LEBAR3B", "MABAL2B", "REPOV2B", "TEBUN1B", "UGEBO1B"],
        "SIDs": ["ANITO8F", "AROSO5F", "DODSO1F", "IDBUD1F", "KIRDA1F", "MASBO5F", "VMR9F", "MIBEL1F", "TAROS1F", "TOMAN5F", 'VOVOS1F']
    },
    "RWY 20C": {
        "STARs":["ARAMA1B", "ASUNA2B", "ELALO1B", "KARTO2B", "LEBAR3B", "MABAL2B", "REPOV2B", "TEBUN1B", "UGEBO1B"],
        "SIDs": ["ANITO8B", "AROSO5B", "DODSO1B", "IDBUD1B", "KIRDA1B", "MASBO5B", "VMR9B", "MIBEL1B", "TAROS1B", "TOMAN5B", "VOVOS1B"]
    },
    "RWY 20L": {
        "STARs":["ARAMA1B", "ASUNA2B", "ELALO1B", "KARTO2B", "LEBAR3B", "MABAL2B", "REPOV2B", "TEBUN1B", "UGEBO1B"],
        "SIDs": ["ANITO1D", "AROSO1D", "DODSO1D", "IDBUD1D", "KIRDA1D", "MASBO1D", "VMR1D", "MIBEL1D", "TAROS1D", "TOMAN1D", "VOVOS1D"]
    },
}

In [ ]:
# fig.write_html(
#     "sg-airnav-map.html",
#     include_plotlyjs='cdn',  # use CDN to reduce file size
#     full_html=True,          # wrap with full HTML boilerplate
#     auto_open=True           # open in browser immediately
# )